In [1]:
import os
import autogen
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders.image import UnstructuredImageLoader
from autogen import register_function
from typing import Annotated, Literal

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [2]:
# def parse_files_in_directory(directory_path):
    
#     parsed_data = parse_pdfs(directory_path)

#     # for filename in os.listdir(directory_path):
#         # file_path = os.path.join(directory_path, filename)
        
#         # if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
#         #     loader = UnstructuredImageLoader(file_path)
#         #     documents = loader.load()
#         #     parsed_text = ' '.join([doc.page_content for doc in documents])

#         # else:
#         #     continue

#         # parsed_data[filename] = parsed_text

#     return parsed_data


def parse_pdfs(directory:str ="../data/") -> dict:
# def parse_pdfs(dir_path="../data/"):
    parsed_data = {}
    loader = PyPDFDirectoryLoader(directory, extract_images=True)
    documents = loader.load()
    for page in documents:
        if parsed_data.get(page.metadata["source"]) is None:
            parsed_data[page.metadata["source"]] = [page.page_content]
        else:
            parsed_data[page.metadata["source"]].append(page.page_content)
    return parsed_data


In [ ]:
api_key = os.environ.get("GROQ_API_KEY")

config_list = [
  {
    "model": "llama-3.2-3b-preview",
    "base_url": "https://api.groq.com/openai/v1/chat/completions",
    "api_type": "groq",
    "api_key": api_key,
    "api_rate_limit": 100
  }
]

# llm_config = config_list[0]
llm_config={"config_list": config_list}

# llm_config={
#     "config_list": config_list,
#     "functions": [
#         {
#             "name": "parse_pdfs",
#             "description": "PDF Parser",
#             "parameters": {
#                 "type": "object",
#                 "properties": {
#                     "directory_path": {
#                         "type": "string",
#                         "description": "path of directory",
#                     },
#                 },
#                 "required": [],
#             },
#         },
#     ],
# }

In [4]:
task = "Parse the documents and classify each of them by a heading"

In [5]:
# user_proxy = autogen.ConversableAgent(
#     name="Admin",
#     system_message="Give the task, and send instructions to doc_classifier to classify the documents in the data folder",
#     code_execution_config=False,
#     llm_config=llm_config,
#     human_input_mode="ALWAYS",
# )

user_proxy = autogen.ConversableAgent(
    name="User",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    # max_consecutive_auto_reply=10,
    # code_execution_config={
    #     "work_dir": "code",
    #     "use_docker": False
    # },
    llm_config=False,
)

In [ ]:
doc_parse_caller = autogen.ConversableAgent(
    name="Doc Parse Caller",
    llm_config=llm_config,
    description="A caller for parsing documents",
    system_message="You are a helpful AI assistant. Suggest how to call the python function parse_pdfs(directory) to parse the directory path at '../data/'."
)

doc_parser = autogen.ConversableAgent(
    name="Doc Parser",
    llm_config=llm_config,
    description="A parser that executes the parser function",
    system_message="You are a doc parser. Execute the python function call suggested to you and return the dictionary."
)

doc_classifier = autogen.ConversableAgent(
    name="Doc Classifier",
    llm_config=llm_config,
    description="A classifier that classifies the text that has been parsed",
    system_message="You are a helpful AI assistant."
                   "For each document provided, categorize them from one of these options ['Passport', 'Residence Permit', 'Transaction', 'Health Insurance', 'Other']."
                   "Reply TERMINATE when the task is done.",
)



In [ ]:
# user_proxy.register_function(
#     function_map={
#         "parse_pdfs": parse_pdfs()
#     }
# )

register_function(
    parse_pdfs,
    caller=doc_parse_caller,  
    executor=doc_parser,  
    name="parse_pdfs",  
    description="A simple document parser",  # A description of the tool.
)

# # Register the tool signature with the assistant agent.
# doc_classifier.register_for_llm(name="parse_pdfs", description="A simple pdf parser")(parse_pdfs)

# # Register the tool function with the user proxy agent.
# user_proxy.register_for_execution(name="parse_pdfs")(parse_pdfs)

<function __main__.parse_pdfs(directory: str = '../data/') -> dict>

In [ ]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, doc_parse_caller, doc_parser, doc_classifier],
    messages=[],
    max_round=5,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [doc_parser],
        doc_parser: [doc_parse_caller, doc_classifier],
        doc_parse_caller: [doc_parser],
        doc_classifier: [user_proxy],
    },
    speaker_transitions_type="allowed",
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [8]:
doc_classifier.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'A simple pdf parser',
   'name': 'parse_pdfs',
   'parameters': {'type': 'object',
    'properties': {'directory': {'type': 'string',
      'default': '../data/',
      'description': 'directory'}},
    'required': []}}}]

In [ ]:
result = user_proxy.initiate_chat(
    doc_classifier,
    # message="Parse the documents and classify each of them by a heading",
    message="Can you classify the parsed documents that you can retreive with the python function I have provide you with?",
)

User (to Classifier):

Can you classify the parsed documents that you can retreive with the python function I have provide you with?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Classifier (to User):

***** Suggested tool call (call_82r3): parse_pdfs *****
Arguments: 
{"directory": "../data/"}
*******************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION parse_pdfs...
User (to Classifier):

User (to Classifier):

***** Response from calling tool (call_82r3) *****
{"..\\data\\ahmad.pdf": ["EUR Statement\nGenerated on the 1 Sept 2023\nPage  of 1 1© 2023 Revolut Bank UAB\nReport lost or stolen card\n+370 5 214 3608\nGet help directly In app\nScan the QR code\nRevolut Bank UAB is a credit institution licensed in the Republic of Lithuania with company number 304580906 and authorisation code \nLB002119, and whose registere

c:\Users\ahmad\Desktop\LlamaHackers\venv\Lib\site-packages\autogen\oai\groq.py:280: UserWarning: Cost calculation not available for model llama-3.2-3b-preview
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Classifier (to User):

***** Suggested tool call (call_mcqc): parse_pdfs *****
Arguments: 
{"directory": "../data/"}
*******************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION parse_pdfs...
User (to Classifier):

User (to Classifier):

***** Response from calling tool (call_mcqc) *****
{"..\\data\\ahmad.pdf": ["EUR Statement\nGenerated on the 1 Sept 2023\nPage  of 1 1© 2023 Revolut Bank UAB\nReport lost or stolen card\n+370 5 214 3608\nGet help directly In app\nScan the QR code\nRevolut Bank UAB is a credit institution licensed in the Republic of Lithuania with company number 304580906 and authorisation code \nLB002119, and whose registered office is at Konstitucijos ave. 21B, LT-08130 Vilnius, the Republic of Lithuania. We are licensed and \nregulated by the Bank of Lithuania and the European Central Bank. The deposits are protected by Lithuanian Deposit Insurance System \nbut s